# Income Optimizer (Table-based)

Simple flow:
1. Load mock tables (`clients`, `client_employees`)
2. Compute user income from `client_employees.monthly_income`
3. Create optimization NBA events for users with income > threshold
4. Optionally post to backend endpoint

In [ ]:
from nba_calculator.utils import CalculatorUtils, EnvConfig

In [ ]:
# Configuration
ENDPOINT = EnvConfig.str("ENDPOINT", "http://127.0.0.1:8000/api/v1/internal/events/nba-calculation")
NBA_LIST_ENDPOINT = EnvConfig.str("NBA_LIST_ENDPOINT", "http://127.0.0.1:8000/api/v1/nba")
THRESHOLD = EnvConfig.float("THRESHOLD", 2000.0)
NBA_DEFINITION_ID = EnvConfig.str("NBA_DEFINITION_ID", "def_optimization")
SOURCE = EnvConfig.str("SOURCE", "calculator.income-threshold.v2")
REQUEST_ID_PREFIX = EnvConfig.str("REQUEST_ID_PREFIX", "calc-income")
DRY_RUN = EnvConfig.bool("DRY_RUN", False)  # Set false to POST events

In [ ]:
def create_event_context(client, employee):
    context ={
            "monthly_income": employee.get("monthly_income"),
            "rule": f"monthly_income > {THRESHOLD}",
            "offer": "optimization",
            "client_id": client.get('client_id'),
            "employee_id": employee.get("employee_id")
            }
    return CalculatorUtils.set_context_hash(context)

def create_delete_event_context(client, employee):
    context = {
        "client_id": client.get('client_id'),
        "employee_id": employee.get("id"),
        "rule": f"monthly_income > {THRESHOLD}",
        "offer": "optimization",
        "message": f"This NBA is no longer applicable to you, since your income is below the threshold.",
        }
    
    return CalculatorUtils.set_context_hash(context)

In [ ]:
# Get all employees that have an income above threshold
query = f"""
INSERT QUERY HERE
"""

client_employees_meeting_criteria = CalculatorUtils.execute_sql_query(query)

In [ ]:
# Find clients employees that already have a linked, active optimization product
query = f"""
INSERT QUERY HERE
"""

client_employees_with_product = CalculatorUtils.execute_sql_query(query)


# Create mapping client_id -> set(employee_id) for employees that already have the product
client_employee_mapping_set = dict()
for client in client_employees_with_product:
    client_id = client.get('client_id')
    employee_id = client.get('employee_id')
    
    tmp = client_employee_mapping_set.get(client_id,set())
    tmp.add(employee_id)
    
    client_employee_mapping_set[client_id] = tmp

In [ ]:
clients_without_product = []

for client in client_employees_meeting_criteria:
    client_id = #...
    employee_id = #...

    # Check if employee_id is in the set of employee_ids for the client_id in the mapping, if not, add to clients_without_product

In [ ]:
new_events = []
for matching_client_employee in clients_without_product:
    client = { key: matching_client_employee.get(key) for key in ["account_id", "client_id", "enterprise_number"]}
    employee = #...

    # Create event context and event for each client-employee that meets the criteria but does not have the product, and add to new_events

In [ ]:
CalculatorUtils.post_events(new_events, dry_run=DRY_RUN, endpoint=ENDPOINT, request_id_prefix=REQUEST_ID_PREFIX)

In the [UI](http://localhost:8000/ui/active-nbas?client=9876543210) you should now see three active NBA's for client with enterprise_number "9876543210"